In [185]:
import sys
import os
import re
import operator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [134]:
# Returns a list of sugar counts if the country is in the tag
def return_sugars(df, country):
    sugar_index = df.columns.get_loc('sugars_100g')
    country_index = df.columns.get_loc('countries_tags')
    return [row[sugar_index] for index, row in df.iterrows() if country in row[country_index]]

In [4]:
# Just the data reading step. I check if the path for the file exist, and if so, ready the data from the file
data_directory = os.getcwd()
data_path = 'data\world-food-facts\FoodFacts.csv'
data_path = os.path.join(data_directory, data_path)

if os.path.exists(data_path): 
    world_food_facts = pd.read_csv(data_path)
    world_food_facts.countries = world_food_facts.countries.str.lower()
    print("Data loaded. There are {} rows".format(world_food_facts.shape[0]))
else:
    print("Error: source data location: {} does not exist".format(data_path))

Data loaded. There are 65503 rows


C:\Users\dsharp\AppData\Local\Continuum\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (0,3,5,27,36) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [138]:
# For my approach, I need to remove the rows where the countries_tags and sugars_100g are null
worlds_sugars = world_food_facts[(world_food_facts.countries_tags.notnull() & world_food_facts.sugars_100g.notnull())]
"After removing rows where countries_tags and sugars_100g columns are null, there are {} rows".format(len(worlds_sugars))

'After removing rows where countries_tags and sugar_100g columns are null, there are 32611'

In [222]:
country_tags = []
for index, item in worlds_sugars.iterrows():
    if len(item[tag_index].split(",")) > 1:
        for country in (item[tag_index].split(",")):
            if ":" in country and country.split(":")[1] not in country_tags:
                country_tags.append(country.split(":")[1])
    else:
        if ":" in country and country.split(":")[1] not in country_tags:
            country_tags.append(country.split(":")[1])

In [223]:
country_tags[:5]

['france', 'united-kingdom', 'united-states', 'australia', 'new-zealand']

In [224]:
len(set(country_tags))

69

In [225]:
num = return_sugars(worlds_sugars, '日本')
print(len(num))
np.mean(return_sugars(worlds_sugars, '日本'))

1


0.0

In [149]:
worlds = world_food_facts[world_food_facts.countries_tags.notnull()]

In [157]:
for country, sugar in zip(worlds.countries_tags, worlds.sugars_100g):
    if 'germany' in country or '日本' in country:
        print(country, sugar)

en:germany 57.5
en:germany 24.0
en:germany nan
en:germany nan
en:germany nan
en:germany nan
en:germany nan
en:germany nan
en:germany nan
en:germany nan
en:germany nan
en:germany nan
en:germany nan
en:germany 15.3
en:germany 3.2
en:germany nan
en:germany 0.0
en:germany 0.0
en:germany 20.0
en:germany 0.0
en:germany nan
en:germany nan
en:germany 7.0
en:germany nan
en:germany 0.0
en:germany nan
en:germany nan
en:germany nan
en:germany nan
en:germany nan
en:germany 10.0
en:belgium,en:france,en:germany,en:netherlands 8.4
en:germany nan
en:germany nan
en:germany nan
en:germany,en:spain 21.9
en:germany nan
en:germany 0.0
en:germany nan
en:germany 4.8
en:germany 4.92
en:germany nan
en:germany nan
en:germany 12.6
en:germany,en:switzerland 0.5
en:germany nan
en:germany,en:switzerland nan
en:germany,en:switzerland nan
en:germany,en:switzerland nan
en:germany,en:switzerland nan
en:germany,en:switzerland nan
en:germany,en:switzerland 87.5
en:germany,en:switzerland 87.5
en:germany,en:switzerland 87.5

In [234]:
sugar_mean_dic = {}
for country in country_tags:
    print("looking for country: {}".format(country))
    sugar = return_sugars(worlds_sugars, country)
    if len(sugar) > 9:
        sugar_mean_dic[country] = np.mean(sugar)

looking for country: france
looking for country: united-kingdom
looking for country: united-states
looking for country: australia
looking for country: new-zealand
looking for country: singapore
looking for country: thailand
looking for country: denmark
looking for country: spain
looking for country: canada
looking for country: quebec
looking for country: czech-republic
looking for country: indonesia
looking for country: italy
looking for country: french-guiana
looking for country: switzerland
looking for country: belgium
looking for country: philippines
looking for country: europe
looking for country: lithuania
looking for country: portugal
looking for country: germany
looking for country: netherlands
looking for country: austria
looking for country: romania
looking for country: scotland
looking for country: morocco
looking for country: ireland
looking for country: guadeloupe
looking for country: monaco
looking for country: reunion
looking for country: luxembourg
looking for country: j

In [235]:
sugar_mean_dic.keys()

dict_keys(['austria', 'hong-kong', 'italy', 'australia', 'romania', 'greece', 'united-states', 'czech-republic', 'reunion', 'spain', 'canada', 'netherlands', 'poland', 'tunisia', 'sweden', 'europe', 'guadeloupe', 'saint-pierre-and-miquelon', 'quebec', 'french-guiana', 'new-zealand', 'portugal', 'france', 'thailand', 'switzerland', 'denmark', 'belgium', 'luxembourg', 'martinique', 'germany', 'united-kingdom'])

In [236]:
sugar_mean_sorted_list = sorted(sugar_mean_dic.items(), key=operator.itemgetter(1))

In [237]:
sugar_mean_sorted_list.reverse()

In [238]:
for country in sugar_mean_sorted_list[:2]:
    print(country[0])

tunisia
quebec


In [239]:
country_list = [pair[0] for pair in sugar_mean_sorted_list]
sugar_list = [pair[1] for pair in sugar_mean_sorted_list]

In [240]:
sugar_list

[28.977894736842106,
 26.186956521739127,
 22.705047619047615,
 22.194210526315789,
 21.314545454545453,
 19.705263157894734,
 18.96153846153846,
 18.307118644067796,
 17.348529411764702,
 16.71658293838863,
 16.605625788999099,
 16.212499999999999,
 15.839976626506024,
 15.402564102564101,
 15.393333333333334,
 15.195345258620689,
 15.111568493150683,
 14.802173913043481,
 14.570942028985508,
 13.665368421052632,
 13.581632653061224,
 13.104297850562947,
 13.103366336633661,
 12.949000000000002,
 12.664607935798456,
 12.527272727272726,
 11.958812500000001,
 10.480405327156921,
 9.6704294478527615,
 8.625,
 8.3478947368421057]

In [241]:
y_pos = np.arange((len(country_list[:10])))
plt.bar(y_pos, sugar_list[:10], align='center', alpha=0.5)
plt.title('Average total sugar content per 100g')
plt.xticks(y_pos, country_list[:10])
plt.ylabel('Sugar/100g')
plt.show()